In [3]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [4]:
import os


os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'advanced-rag'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")

In [6]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_groq import ChatGroq
from langchain_community.embeddings.huggingface import HuggingFaceBgeEmbeddings


#### INDEXING ####

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

##1 - 0 - 1000 , 800 - 1800

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
from langchain_community.embeddings import OllamaEmbeddings

# Split - Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


# For HuggingFaceBGEEmbeddings Embed
# model_name = "BAAI/bge-small-en-v1.5"
# model_kwargs = {"device": "cpu"}
# encode_kwargs = {"normalize_embeddings": True}

# hf_embeddings = HuggingFaceBgeEmbeddings(
#     model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
# )

ollama_embeddings = OllamaEmbeddings(model="llama3.1")
vectorstore = FAISS.from_documents(documents=splits, 
                                    embedding=ollama_embeddings)

retriever = vectorstore.as_retriever() # Dense Retrieval - Embeddings/Context based

C:\Users\praka\AppData\Local\Temp\ipykernel_21936\1924305968.py:17: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  ollama_embeddings = OllamaEmbeddings(model="llama3.1")


In [8]:
#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatGroq(model="llama3-8b-8192", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [9]:
# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
print(rag_chain.invoke("What is Task Decomposition?"))

I don't know. The provided context does not mention Task Decomposition.


## Part 2 Indexing

In [10]:
# Documents
question = "What kinds of pets do I like?"
document = "My favorite pet is a cat."

### Document Loaders

In [11]:
#### INDEXING ####

# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

### Splitter

In [12]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, 
    chunk_overlap=200)

# Make splits
splits = text_splitter.split_documents(blog_docs)

### Text embedding models

In [13]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
# model_name = str("BAAI/bge-small-en")
# model_kwargs = {"device": str("cpu")}
# encode_kwargs = {"normalize_embeddings": bool(True)}
# hf_embeddings = HuggingFaceBgeEmbeddings(
#     model_name=str(model_name), model_kwargs=dict(model_kwargs), encode_kwargs=dict(encode_kwargs)
# )

ollama_embeddings = OllamaEmbeddings(model="llama3.1")
query_result = ollama_embeddings.embed_query(question)
document_result = ollama_embeddings.embed_query(document)
len(query_result)

4096

In [14]:
import numpy as np

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_product / (norm_vec1 * norm_vec2)

similarity = cosine_similarity(query_result, document_result)
print("Cosine Similarity:", similarity)

Cosine Similarity: 0.69559268457881


### Vectorstores

In [15]:
# Index
from langchain_community.vectorstores import FAISS
vectorstore = FAISS.from_documents(documents=splits, 
                                    embedding=ollama_embeddings)

retriever = vectorstore.as_retriever()

In [16]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000017F67635C30>, search_kwargs={})

## Part 3: Retrieval

In [17]:
docs = retriever.get_relevant_documents("What is Task Decomposition?")

C:\Users\praka\AppData\Local\Temp\ipykernel_21936\4059233835.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("What is Task Decomposition?")


In [18]:
len(docs)

4

In [19]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 9. Comparison of MIPS algorithms, measured in recall@10. (Image source: Google Blog, 2020)\nCheck more MIPS algorithms and performance comparison in ann-benchmarks.com.\nComponent Three: Tool Use#\nTool use is a remarkable and distinguishing characteristic of human beings. We create, modify and utilize external objects to do things that go beyond our physical and cognitive limits. Equipping LLMs with external tools can significantly extend the model capabilities.\n\nFig. 10. A picture of a sea otter using rock to crack open a seashell, while floating in the water. While some other animals can use tools, the complexity is not comparable with humans. (Image source: Animals using tools)\nMRKL (Karpas et al. 2022), short for “Modular Reasoning, Knowledge and Language”, is a neuro-symbolic architecture for autonomous agents. A MRKL system is proposed to contain a collection of “expert” 

## Part 4: Generation

In [20]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate

# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])

In [21]:
# LLM
llm = ChatGroq(model="llama3-8b-8192", temperature=0)

In [22]:
# Chain
chain = prompt | llm

In [23]:
# Run
chain.invoke({"context":docs,"question":"What is Task Decomposition?"})

AIMessage(content='According to the provided context, Task Decomposition is the process of breaking down a complicated task into smaller, manageable subgoals, enabling efficient handling of complex tasks. This is mentioned in the section "Component One: Planning" of the document, where it is described as a crucial aspect of planning in a LLM-powered autonomous agent system.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 68, 'prompt_tokens': 2932, 'total_tokens': 3000, 'completion_time': 0.056666667, 'prompt_time': 0.132537852, 'queue_time': 0.00388322499999999, 'total_time': 0.189204519}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run-cb0205ec-3d07-4591-bafb-a8402ccf4077-0', usage_metadata={'input_tokens': 2932, 'output_tokens': 68, 'total_tokens': 3000})

### RAG chains

In [24]:
from langchain import hub
prompt_hub_rag = hub.pull("rlm/rag-prompt")

In [25]:
prompt_hub_rag

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt_hub_rag
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

"I don't know. The provided context does not mention Task Decomposition."